In [19]:
#!pip install opencv-python
#!pip install scikit-image

from skimage import color, feature
from skimage.feature import hog
from skimage.io import imread
import numpy as np
import cv2
import matplotlib.pyplot as plt
from matplotlib import image as mpimg
import os
import glob
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

ModuleNotFoundError: No module named 'skimage'

In [ ]:
from skimage import color, feature
print("Scikit-image se ha importado correctamente.")

## **Anàlisi del atributs**

### **Càrrega i visualització d'una imatge**
Primer, carreguem una imatge del dataset utilitzant la funció `mpimg.imread`, que converteix la imatge en un array de NumPy per poder treballar-hi. Aquesta imatge pertany a la carpeta de "freshapples". Un cop carregada, habilitem l'opció d'escriure sobre l'array amb `setflags(write=1)` per permetre modificacions en el futur.

Després, imprimim les dimensions de la imatge (`img.shape`) per confirmar la seva resolució i nombre de canals (normalment 3 si és en color). Finalment, visualitzem la imatge amb `plt.imshow()` per assegurar-nos que es veu correctament i per familiaritzar-nos amb les dades abans de començar el processament.


In [ ]:


img=np.array(mpimg.imread('.gitignore/dataset/train/freshapples/rotated_by_15_Screen Shot 2018-06-08 at 4.59.36 PM.png'))
img.setflags(write=1)
print('Image : ',img.shape)
plt.imshow(img)
plt.show()

### **Redimensionament i visualització d'una imatge**
Redimensionem la imatge carregada prèviament utilitzant la funció `cv2.resize`. Ajustem les seves dimensions a 150x150 píxels per reduir la mida de les dades i optimitzar el processament.

Després del redimensionament, mostrem la imatge redimensionada amb `plt.imshow()` per verificar que s'ha aplicat correctament. Finalment, utilitzem `print(resized_img.shape)` per comprovar que les dimensions resultants són les esperades (150x150 píxels).


In [ ]:
import cv2

# Redimensionar la imatge
resized_img = cv2.resize(img, (150, 150))

# Mostrar la imatge redimensionada
plt.imshow(resized_img)
plt.show()
print(resized_img.shape)

### **Extracció de característiques amb HOG**
A continuació, convertim la imatge a RGB seleccionant només aquests canals i eliminem l'eventual canal alfa de la imatge (`resized_img[:, :, :3]`).

Després, convertim la imatge a una escala de grisos utilitzant la funció `color.rgb2gray`, eliminant la complexitat dels colors per centrar-nos en l'estructura i els detalls importants.

Posteriorment, calculem les característiques HOG (Histogram of Oriented Gradients) amb la funció `feature.hog`. Aquesta tècnica ens permet detectar formes i textures a partir dels gradients de la imatge. També activem l'opció de visualització (`visualize=True`) per obtenir una representació visual del HOG.

Finalment, imprimim les dimensions i els valors de les característiques HOG (`fd`) i mostrem la imatge HOG amb `plt.imshow`, utilitzant un mapa de color en escala de grisos (`cmap="gray"`).

```python
print(fd.shape)       # Mostrem la forma de les característiques HOG
print(fd)             # Mostrem les característiques HOG
print(hog_image.shape)  # Mostrem la forma de la imatge HOG


In [ ]:
from skimage import color, feature
# Remove the alpha channel from the image
rgb_img = resized_img[:, :, :3]


# Convert the image to grayscale
gray_img = color.rgb2gray(rgb_img)

# Compute HOG features for the grayscale image
fd, hog_image = feature.hog(gray_img, orientations=9, pixels_per_cell=(8, 8),
                            cells_per_block=(2, 2), visualize=True)

print(fd.shape)
print(fd)
print(hog_image.shape)
# Display the HOG image
plt.axis("off")
plt.imshow(hog_image, cmap="gray")

### **Processament i anàlisi del dataset**

En aquest codi processem el dataset de fruites (fresques i podrides), extraient característiques amb HOG i preparant les dades per a la classificació.

#### **Processament del dataset**
1. Carreguem les imatges de les carpetes corresponents (`freshapples`, `rottenapples`, etc.).
2. Preprocessem cada imatge:
   - Redimensionem a 150x150 píxels.
   - Convertim a escala de grisos.
   - Extraiem característiques HOG per identificar patrons visuals (bordes i textures).
3. Guardem les característiques a `data` i les etiquetes a `labels`, comptant les imatges per categoria.

#### **Visualització**
Generem un gràfic de barres per mostrar la distribució d’imatges per categoria i verificar que el dataset està equilibrat.

```python
plt.bar(fruit_labels, image_counts, color=['green', 'yellow', 'orange', 'red', 'purple', 'brown'])
plt.title('Distribució de les Imatges')
plt.xlabel('Tipus de Fruita')
plt.ylabel('Número d’Imatges')
plt.show()


In [ ]:
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import hog

# Inicializar listas globales para características y etiquetas
data = []
labels = []

# Definir rutas y etiquetas correspondientes
freshfruits = {
    "freshapples": "freshapple",
    "freshbanana": "freshbanana",
    "freshoranges": "freshorange"
}

rottenfruits = {
    "rottenapples": "rottenapple",
    "rottenbanana": "rottenbanana",
    "rottenoranges": "rottenorange"
}

# Inicializar contadores para cada categoría
fruit_counts = {
    "freshapple": 0,
    "freshbanana": 0,
    "freshorange": 0,
    "rottenapple": 0,
    "rottenbanana": 0,
    "rottenorange": 0
}

# Procesar cada tipo de fruta frescas
for fruit_folder, label in freshfruits.items():
    folder_path = f".gitignore/dataset/train/{fruit_folder}/*.png"
    for entry in glob.glob(folder_path):
        img = cv2.imread(entry)  # Leer imagen
        resized_img = cv2.resize(img, (150, 150))  # Redimensionar imagen
        gray_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)  # Convertir a escala de grises
        fd = hog(gray_img)  # Calcular características HOG
        data.append(fd)  # Agregar características
        labels.append(label)  # Agregar etiqueta correspondiente
        fruit_counts[label] += 1  # Contar imagen por etiqueta
    print(f"{label.capitalize()} Images: {fruit_counts[label]}")

# Procesar cada tipo de fruta no frescas
for fruit_folder, label in rottenfruits.items():
    folder_path = f".gitignore/dataset/train/{fruit_folder}/*.png"
    for entry in glob.glob(folder_path):
        img = cv2.imread(entry)  # Leer imagen
        resized_img = cv2.resize(img, (150, 150))  # Redimensionar imagen
        gray_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)  # Convertir a escala de grises
        fd = hog(gray_img)  # Calcular características HOG
        data.append(fd)  # Agregar características
        labels.append(label)  # Agregar etiqueta correspondiente
        fruit_counts[label] += 1  # Contar imagen por etiqueta
    print(f"{label.capitalize()} Images: {fruit_counts[label]}")

# Verificar la longitud de los datos y las etiquetas
print("Total number of images: " + str(len(data)))
print("Total number of labels: " + str(len(labels)))

# Datos para el gráfico
fruit_labels = list(fruit_counts.keys())
image_counts = list(fruit_counts.values())

# Crear gráfico de barras
plt.figure(figsize=(10,6))
plt.bar(fruit_labels, image_counts, color=['green', 'yellow', 'orange', 'red', 'purple', 'brown'])

# Añadir títulos y etiquetas
plt.title('Distribución de Imágenes de Frutas (Frescas y Podridas)', fontsize=14)
plt.xlabel('Tipo de Fruta', fontsize=12)
plt.ylabel('Número de Imágenes', fontsize=12)

# Mostrar el gráfico
plt.xticks(rotation=45)
plt.tight_layout()  # Ajusta el diseño para que las etiquetas no se solapen
plt.show()


### **Entrenament i avaluació del model**

Aquest codi entrena un model SVM amb l’estratègia One-vs-One per classificar el tipus de fruita (`apple`, `banana`, `orange`), independentment de si està fresca o podrida.

#### **Procés**
1. Convertim les dades i etiquetes a arrays de NumPy i normalitzem les etiquetes per identificar només el tipus de fruita.
2. Codifiquem les etiquetes amb `LabelEncoder` i dividim les dades en entrenament (70%) i prova (30%) amb estratificació.
3. Configurem i entrenem el model SVM amb kernel lineal.
4. Avaluem el model amb l'exactitud (`accuracy_score`) i un informe de classificació (`classification_report`).

#### **Objectiu**
Identificar el tipus de fruita amb alta precisió i verificar el rendiment amb les dades de prova.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
import glob

# Convertir los datos y etiquetas a arrays
data = np.array(data)

# Suponiendo que ya tienes las listas `freshfruits` y `rottenfruits`
# Crear las etiquetas originales a partir de las carpetas de frutas frescas y podridas
original_labels = []

# Etiquetas de frutas frescas
for fruit_folder, label in freshfruits.items():
    folder_path = f".gitignore/dataset/train/{fruit_folder}/*.png"
    for entry in glob.glob(folder_path):
        original_labels.append(label)

# Etiquetas de frutas podridas
for fruit_folder, label in rottenfruits.items():
    folder_path = f".gitignore/dataset/train/{fruit_folder}/*.png"
    for entry in glob.glob(folder_path):
        original_labels.append(label)

# Normalizar las etiquetas para que sean solo tipos de frutas (ignorando si están frescas o podridas)
def normalize_label(label):
    if "apple" in label:
        return "apple"
    elif "banana" in label:
        return "banana"
    elif "orange" in label:
        return "orange"
    return label  # En caso de que no coincida con las opciones anteriores

# Normalizar las etiquetas antes de codificarlas
normalized_labels = np.array([normalize_label(label) for label in original_labels])

# Codificar etiquetas como valores numéricos
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(normalized_labels)

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(data, encoded_labels, test_size=0.3, random_state=42, stratify=encoded_labels)

# Configurar el modelo con la estrategia One-vs-One
ovo_classifier = OneVsOneClassifier(SVC(kernel='linear', probability=True, random_state=42))

# Entrenar el modelo
ovo_classifier.fit(X_train, y_train)

# Evaluar el modelo
y_pred = ovo_classifier.predict(X_test)

# Mostrar resultados
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=encoder.classes_))


### **Entrenament i avaluació del model**

Utilitzem un model SVM amb estratègia One-vs-Rest per classificar el tipus de fruita (`apple`, `banana`, `orange`).

#### **Procés**
1. Normalitzem i codifiquem les etiquetes per identificar només el tipus de fruita.
2. Dividim les dades en entrenament (70%) i prova (30%).
3. Entrenem un model SVM amb kernel lineal.
4. Avaluem l'exactitud i generem un informe de classificació.

#### **Objectiu**
Comparar el rendiment del model amb l'estratègia One-vs-One.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Convertir los datos y etiquetas a arrays
data = np.array(data)
labels = np.array(labels)

original_labels = []
for fruit_folder, label in freshfruits.items():
    folder_path = f".gitignore/dataset/train/{fruit_folder}/*.png"
    for entry in glob.glob(folder_path):
        original_labels.append(label)

for fruit_folder, label in rottenfruits.items():
    folder_path = f".gitignore/dataset/train/{fruit_folder}/*.png"
    for entry in glob.glob(folder_path):
        original_labels.append(label)

# Normalizar las etiquetas para que sean solo tipos de frutas (ignorando si están frescas o podridas)
def normalize_label(label):
    if "apple" in label:
        return "apple"
    elif "banana" in label:
        return "banana"
    elif "orange" in label:
        return "orange"

# Normalizar las etiquetas antes de codificarlas
normalized_labels = np.array([normalize_label(label) for label in original_labels])

# Codificar etiquetas como valores numéricos
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(normalized_labels)

# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(data, encoded_labels, test_size=0.3, random_state=42, stratify=encoded_labels)

# Configurar el modelo con la estrategia One-vs-Rest
ovr_classifier = OneVsRestClassifier(SVC(kernel='linear', probability=True, random_state=42))

# Entrenar el modelo
ovr_classifier.fit(X_train, y_train)

# Evaluar el modelo
y_pred = ovr_classifier.predict(X_test)

# Mostrar resultados
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=encoder.classes_))


### **Anàlisi dels resultats**

Els resultats de l’estratègia One-vs-Rest són similars als de l’estratègia One-vs-One. Per aprofundir en l’anàlisi, provarem diferents tipus de kernel en el model SVM per observar com varien els resultats i determinar si algun kernel ofereix una millora significativ


### **Classificació amb Kernel RBF**

Provem un model SVM amb kernel RBF (`Radial Basis Function`) per veure com afecta la no linealitat als resultats de classificació.

#### **Característiques rellevants**
- **Gamma:** Configurat a `scale`, ajusta automàticament la influència dels punts de dades individuals.
- **Adaptació a no linealitats:** El kernel RBF permet capturar relacions més complexes en les dades.

#### **Objectiu**
Analitzar si el kernel RBF millora el rendiment respecte al kernel lineal utilitzat anteriorment.


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(data, encoded_labels, test_size=0.3, random_state=42, stratify=encoded_labels)

# Configurar el modelo con el kernel RBF
rbf_classifier = SVC(kernel='rbf', gamma='scale', probability=True, random_state=42)

# Entrenar el modelo
rbf_classifier.fit(X_train, y_train)

# Hacer predicciones
y_pred = rbf_classifier.predict(X_test)

# Mostrar resultados
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=encoder.classes_))


### **Classificació amb Kernel Polinòmic**

Provem un model SVM amb kernel polinòmic per avaluar com afecta aquest tipus de kernel als resultats de classificació.

#### **Característiques rellevants**
- **Degree:** Configurat a 3, defineix el grau del polinomi i la complexitat de les relacions que pot modelar.
- **Coef0:** Configurat a 1, controla l’impacte dels termes independents del polinomi.
- **Aplicació:** Ideal per capturar relacions moderadament no lineals en les dades.

#### **Objectiu**
Determinar si el kernel polinòmic, amb un grau fixat a 3, ofereix una millora respecte al kernel lineal i RBF.


In [ ]:
# Configurar el modelo con el kernel polinómico
poly_classifier = SVC(kernel='poly', degree=3, coef0=1, probability=True, random_state=42)

# Entrenar el modelo
poly_classifier.fit(X_train, y_train)

# Hacer predicciones
y_pred = poly_classifier.predict(X_test)

# Mostrar resultados
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=encoder.classes_))


Provem amb models preentrenats:

### **Classificació amb Transfer Learning utilitzant VGG16**

En aquest codi, implementem un model de classificació de fruites utilitzant transfer learning amb l’arquitectura preentrenada VGG16.

#### **Característiques clau**
- **VGG16 preentrenat:** S’utilitza com a base amb els pesos d’ImageNet, eliminant la capa superior (`include_top=False`) per adaptar-lo al nostre problema.
- **Capacitat d'aprenentatge congelada:** Les capes de VGG16 es mantenen no entrenables (`layer.trainable = False`) per aprofitar les característiques preentrenades.
- **Cap personalitzada:** S’afegeixen capes al final:
  - `GlobalAveragePooling2D`: Per reduir la dimensionalitat.
  - `Dense` amb 1024 unitats i activació `relu` per afegir complexitat.
  - `Dense` amb 6 unitats i activació `softmax` per a la classificació de les 6 classes.

#### **Generació de dades**
- Utilitzem `ImageDataGenerator` per generar dades augmentades:
  - Escalem els píxels a un rang [0, 1].
  - Apliquem transformacions com rotacions, desplaçaments, zooms i reflexions horitzontals per evitar sobreajustament.
- Les dades es carreguen en carpetes separades per a entrenament i validació.

#### **Entrenament**
- El model es compila amb `categorical_crossentropy` com a funció de pèrdua i `Adam` com a optimitzador.
- Entrenem durant 10 èpoques utilitzant les dades augmentades.

#### **Objectiu**
Aprofitar el transfer learning amb VGG16 per millorar la classificació multiclasse de fruites i comparar els resultats amb els models SVM anteriors.


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Ruta a las carpetas de entrenamiento y validación
train_dir = '.gitignore/dataset/train'
test_dir = '.gitignore/dataset/test'

# Usar VGG16 preentrenado sin la capa superior
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Congelar las capas del modelo base
for layer in vgg_base.layers:
    layer.trainable = False

# Añadir nuestras capas personalizadas
x = vgg_base.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(6, activation='softmax')(x)  # 6 clases en total: freshapple, freshbanana, freshorange, rottenapple, rottenbanana, rottenorange

# Crear el modelo final
model = Model(inputs=vgg_base.input, outputs=x)

# Compilar el modelo
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Crear generadores de datos para entrenar y validar
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Escalar los píxeles a un rango [0, 1]
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Cargar las imágenes de entrenamiento
train_generator = train_datagen.flow_from_directory(
    train_dir,  # Ruta de entrenamiento
    target_size=(224, 224),  # Tamaño para VGG16
    batch_size=32,
    class_mode='categorical'  # Como es clasificación multiclase
)

# Cargar las imágenes de validación
validation_generator = val_datagen.flow_from_directory(
    test_dir,  # Ruta de prueba
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Entrenar el modelo
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

# Guardar el modelo entrenado
model.save('fruit_classification_model.h5')

### **Avaluació del model amb dades de prova**

En aquest codi, avaluem el model entrenat utilitzant el conjunt de dades de prova i generem mètriques per analitzar-ne el rendiment.

#### **Característiques clau**
- **Generador de dades de validació:** Es carrega el conjunt de prova amb `ImageDataGenerator` i es manté l'ordre de les etiquetes (`shuffle=False`) per garantir que les prediccions coincideixin amb les etiquetes reals.
- **Prediccions:** Utilitzem el model carregat (`fruit_classification_model.h5`) per generar prediccions sobre el conjunt de prova.
- **Mètriques d’avaluació:**
  - **Informe de classificació (`classification_report`):** Mostra mètriques com precisió, recall i F1-score per cada classe.
  - **Matriiu de confusió:** Identifica on es produeixen errors en les prediccions, visualitzant les etiquetes reals vs. prediccions.

#### **Visualització**
- La **matriz de confusió** es representa gràficament amb un mapa de calor (`heatmap`) que facilita identificar les classes amb millor i pitjor rendiment.

#### **Objectiu**
Avaluar la precisió i consistència del model en dades reals de prova i identificar possibles millores per optimitzar el rendiment.


In [ ]:
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Directorio de prueba
test_dir = '.gitignore/dataset/test'

# Crear generador de datos para validación
val_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = val_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # Importante para mantener el orden de las etiquetas
)

# Cargar el modelo entrenado
model = tf.keras.models.load_model('fruit_classification_model.h5')

# Generar predicciones con el conjunto de validación
validation_generator.reset()  # Reiniciar el generador
predictions = model.predict(validation_generator, steps=validation_generator.samples // validation_generator.batch_size + 1)
predicted_classes = np.argmax(predictions, axis=1)

# Obtener las etiquetas reales
true_classes = validation_generator.classes
class_labels = list(validation_generator.class_indices.keys())

# Generar el informe de clasificación
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print("Informe de Clasificación:")
print(report)

# Matriz de confusión
conf_matrix = confusion_matrix(true_classes, predicted_classes)

# Visualizar la matriz de confusión
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
plt.title('Matriz de Confusión')
plt.ylabel('Etiqueta Real')
plt.xlabel('Etiqueta Predicha')
plt.show()


In [21]:
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Configuración del dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando dispositivo: {device}")


ModuleNotFoundError: No module named 'torch'